In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import glob

def psnr_metric(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=1)
    psnr_1 = tf.image.psnr(y_true, y_pred, max_val=1)
    return tf.reduce_mean(psnr_1)

def ssim_metric(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=1)
    ssim_1 = tf.image.ssim(y_true, y_pred, max_val=1)
    return tf.reduce_mean(ssim_1)

# Build the model:
inputs = tf.keras.Input(shape=(None,None,1))
inputs_conv = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(inputs)
conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(inputs_conv)
conv1 = tf.keras.layers.ReLU()(conv1)
conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(conv1)
conv2 = tf.keras.layers.ReLU()(conv2)
conv3 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(conv2)
conv3 = tf.keras.layers.ReLU()(conv3)
conv4 = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(conv3)
conv4 = tf.keras.layers.ReLU()(conv4)

# Feature Fusion
mix_global = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same')(conv4)
add_global = tf.keras.layers.Add()([mix_global, inputs_conv])
features = tf.keras.layers.Conv2D(filters=4, kernel_size=3, padding='same')(add_global)
outputs = tf.nn.depth_to_space(features, 2)

# Defining the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Defining Adam1
Adam1 = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Load weights
# model.load_weights("/content/model.h5")

# Compile the model.
model.compile(optimizer=Adam1, loss=tf.keras.losses.MeanAbsoluteError(), metrics=[psnr_metric, ssim_metric])
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Copy data from google drive
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/Datasets/Manga109_Train_LR.zip /content/LR.zip
!cp /content/drive/MyDrive/Datasets/Manga109_Train_HR.zip /content/HR.zip

!unzip /content/LR.zip
!unzip /content/HR.zip

In [ ]:
# Load data into memory
import cv2
filelist1 = sorted(glob.glob('/content/LR/*.png'))
train_in = []
for myFile in filelist1:
    image = cv2.imread(myFile, cv2.IMREAD_GRAYSCALE)
    train_in.append(image)
train_in = np.array(train_in).astype(np.float32) / 255.0
train_in = tf.clip_by_value(train_in, clip_value_min=0, clip_value_max=1)
train_in = np.expand_dims(train_in, axis=3) # comment this for RGB

filelist2 = sorted(glob.glob('/content/HR/*.png'))
train_ref = []
for myFile in filelist2:
    image = cv2.imread(myFile, cv2.IMREAD_GRAYSCALE)
    train_ref.append(image)
train_ref = np.array(train_ref).astype(np.float32) / 255.0
train_ref = tf.clip_by_value(train_ref, clip_value_min=0, clip_value_max=1)
train_ref = np.expand_dims(train_ref, axis=3) # comment this for RGB

In [ ]:
# Train the model.
history = model.fit(train_in, train_ref, epochs=1000, batch_size=64, verbose=1)

In [ ]:
# Make a single prediction
input_bgr = cv2.imread('/content/input.png', cv2.IMREAD_COLOR)
input_bgr = np.array(input_bgr).astype(np.float32) / 255.0
input_bgr = np.clip(input_bgr, 0, 1)

height, width, channels = input_bgr.shape

input_ycrcb = cv2.cvtColor(input_bgr, cv2.COLOR_BGR2YCrCb, 0)
(input_y, input_cr, input_cb) = cv2.split(input_ycrcb)

input_y = np.expand_dims(input_y, axis=0)
input_y = np.expand_dims(input_y, axis=3)

pred_y = model.predict(input_y)
pred_y = np.clip(pred_y, 0, 1)

pred_y = np.squeeze(pred_y, axis=0)
pred_y = np.squeeze(pred_y, axis=2)

inter_cr = cv2.resize(input_cr, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
inter_cb = cv2.resize(input_cb, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)

pred_ycrcb = np.empty([height * 2, width * 2, channels]).astype(np.float32)
pred_ycrcb[:,:,0] = pred_y
pred_ycrcb[:,:,1] = inter_cr
pred_ycrcb[:,:,2] = inter_cb

pred_ycrcb = np.clip(pred_ycrcb, 0, 1)

pred_bgr = cv2.cvtColor(pred_ycrcb, cv2.COLOR_YCrCb2BGR, 0)

pred_bgr = np.clip(pred_bgr, 0, 1)
pred_bgr = pred_bgr * 255
pred_bgr = np.squeeze((np.around(pred_bgr)).astype(np.uint8))

cv2.imwrite('/content/prediction.png', pred_bgr)

In [ ]:
model.save('/content/model.h5')
model.save('/content/model/')
!zip -r model.zip '/content/model/'

In [ ]:
print(history.history['loss'])

In [ ]:
print(history.history['psnr_metric'])

In [ ]:
print(history.history['ssim_metric'])

In [ ]:
!rm /content/model.zip
!rm -r /content/content/
!rm -r /content/model/